![Capgemini](https://raw.githubusercontent.com/interviewquery/takehomes/capgemini_3/capgemini_3/logo.png)
# Purpose

The team has asked that you prepare a 20 to 30-minute presentation (in this notebook) on one of the topics from below. The purpose of this
exercise is to demonstrate your ability to draw insights from data, put
insights in business-friendly format and confirm coding knowledge. These
topics are similar in nature to projects we run. Please make sure that
your presentation is accessible to a general technical audience (aside from the sections of code, of course).


# Data


**Sales Forecasting**

Build and evaluate models to predict national retail store sales for
each store and department. Our sales are very seasonal, and we make
our money during holidays like Super Bowl, Labor Day, Thanksgiving,
and Christmas. The data is contained in `sales_forcasting.csv`

# Deliverables
Your notebook should contain:

1.  Description of the problem; state what you are solving/ analyzing
2.  Presentation of insights/conclusion you generate
3.  Relevant descriptive statistics (charts, graphs, etc.)
4.  Specification of predictive model (mathematical formulation)
5.  Relevant model diagnostics
6.  Model interpretation (what do the coefficients mean, how do you use
    them?)
7.  Please specify language, packages and libraries used to develop your
    solution

# Evaluation Criteria

- Presentation on analysis conducted that covers business outcomes and statistical methodologies
- Preform exploratory data analysis to gather starting insights and conclusions
- Selection of ML/Predictive modeling technique(s) & feature extraction
- Knowledge with data ingestion tools/languages
- Ability to conduct appropriate data cleansing if any
- Ability to code in open source languages

# Rules

- Use any open source language of your choice

- Solution you provide should be your own. Reference any material desired.

- Be prepared to discuss your code in-depth (what it does, how it does it etc.)

- Utilize any statistical or ML technique(s) you deem relevant. For
each technique that you use, be prepared to talk about model
diagnostics, results, and mathematics behind your technique(s).

- No time limit on developing your solution. Let us know when you are ready.


In [1]:
!git clone --branch capgemini_3 https://github.com/interviewquery/takehomes.git
%cd takehomes/capgemini_3
!ls

Cloning into 'takehomes'...
remote: Enumerating objects: 1963, done.
remote: Counting objects: 100% (1963/1963), done.
remote: Compressing objects: 100% (1220/1220), done.
remote: Total 1963 (delta 752), reused 1927 (delta 726), pack-reused 0 (from 0)
Receiving objects: 100% (1963/1963), 297.43 MiB | 12.64 MiB/s, done.
Resolving deltas: 100% (752/752), done.
/content/takehomes/capgemini_3
logo.png  metadata.json  sales_forecasting.csv	takehomefile.ipynb


In [20]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from xgboost import cv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error
import plotly.graph_objects as go
os.listdir()

['sales_forecasting.csv', 'metadata.json', 'takehomefile.ipynb', 'logo.png']

# READ DATA

In [3]:
# read data
df = pd.read_csv("sales_forecasting.csv")

# print
print(df.shape)
df.head()

(421570, 5)


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


# EPLORATIVE ANALYSIS

In [58]:
# NaA checking
for str_col in df.columns.tolist():
  n_nan = df["IsHoliday"].isna().sum()
  print(str_col + " has NaN values: " + str(n_nan))


Store has NaN values: 0
Dept has NaN values: 0
Date has NaN values: 0
Weekly_Sales has NaN values: 0
IsHoliday has NaN values: 0


In [51]:
# general overview
d_dict_formula = {"Dept":"nunique",
                  "Date":["min", "max"],
                  "Weekly_Sales":["min", "mean", "max", "std"]
                  }
df.groupby(["Store"], as_index=False).agg(d_dict_formula)

Store    Dept        Date             Weekly_Sales                \
         nunique         min         max          min          mean   
0      1      77  2010-02-05  2012-10-26      -863.00  21710.543621   
1      2      78  2010-02-05  2012-10-26     -1098.00  26898.070031   
2      3      72  2010-02-05  2012-10-26     -1008.96   6373.033983   
3      4      78  2010-02-05  2012-10-26      -898.00  29161.210415   
4      5      72  2010-02-05  2012-10-26      -101.26   5053.415813   
5      6      77  2010-02-05  2012-10-26      -698.00  21913.243624   
6      7      76  2010-02-05  2012-10-26      -459.00   8358.766148   
7      8      76  2010-02-05  2012-10-26      -100.00  13133.014768   
8      9      73  2010-02-05  2012-10-26      -496.00   8772.890379   
9     10      77  2010-02-05  2012-10-26      -798.00  26332.303819   
10    11      78  2010-02-05  2012-10-26      -594.00  19276.762751   
11    12      75  2010-02-05  2012-10-26      -598.00  14867.308619   
12    13      79  2010-02-05  2012-10-26       -98.00  27355.136891   
13    14      77  2010-02-05  2012-10-26      -498.00  28784.851727   
14    15      79  2010-02-05  2012-10-26      -179.00   9002.493073   
15    16      76  2010-02-05  2012-10-26     -1699.00   7863.224124   
16    17      76  2010-02-05  2012-10-26      -259.00  12954.393636   
17    18      78  2010-02-05  2012-10-26      -259.00  15733.313136   
18    19      79  2010-02-05  2012-10-26      -649.00  20362.126734   
19    20      78  2010-02-05  2012-10-26      -798.00  29508.301592   
20    21      77  2010-02-05  2012-10-26      -450.00  11283.435496   
21    22      76  2010-02-05  2012-10-26      -175.54  15181.218886   
22    23      77  2010-02-05  2012-10-26      -298.00  19776.180881   
23    24      77  2010-02-05  2012-10-26      -109.00  18969.106500   
24    25      77  2010-02-05  2012-10-26       -99.87  10308.157810   
25    26      76  2010-02-05  2012-10-26       -79.00  14554.129672   
26    27      78  2010-02-05  2012-10-26      -409.00  24826.984536   
27    28      77  2010-02-05  2012-10-26     -4988.94  18714.889803   
28    29      77  2010-02-05  2012-10-26      -300.00   8158.810609   
29    30      64  2010-02-05  2012-10-26       -26.61   8764.237719   
30    31      76  2010-02-05  2012-10-26      -224.00  19681.907464   
31    32      77  2010-02-05  2012-10-26      -698.00  16351.621855   
32    33      63  2010-02-05  2012-10-26       -39.76   5728.414053   
33    34      78  2010-02-05  2012-10-26      -598.00  13522.081671   
34    35      74  2010-02-05  2012-10-26     -3924.00  13803.596986   
35    36      63  2010-02-05  2012-10-26      -419.28   8584.412563   
36    37      62  2010-02-05  2012-10-26       -64.38  10297.355026   
37    38      63  2010-02-05  2012-10-26      -153.12   7492.478460   
38    39      75  2010-02-05  2012-10-26      -159.00  21000.763562   
39    40      77  2010-02-05  2012-10-26      -543.00  13763.632803   
40    41      77  2010-02-05  2012-10-26      -279.00  17976.004648   
41    42      62  2010-02-05  2012-10-26      -379.00  11443.370118   
42    43      61  2010-02-05  2012-10-26      -372.90  13415.114118   
43    44      62  2010-02-05  2012-10-26      -132.46   6038.929814   
44    45      74  2010-02-05  2012-10-26     -1750.00  11662.897315   

                             
          max           std  
0   203670.47  27748.945511  
1   285353.53  33077.612059  
2   155897.94  14251.034807  
3   385051.04  34583.677814  
4    93517.72   8068.221050  
5   342578.65  23633.427075  
6   222921.09  10679.008085  
7   153431.69  15132.069598  
8   139427.43  12446.502614  
9   693099.36  32133.006264  
10  245767.47  20391.897907  
11  360140.66  17967.248878  
12  292165.78  33853.474638  
13  474330.10  36911.116744  
14  292555.25  11362.009143  
15  129372.97  10491.369269  
16  194660.52  16299.024131  
17  353008.64  18556.351564  
18  339846.03  24281.125431  
19  422306.25  35661.530620  
20  2

## store

In [96]:
# general overview of the ts per each store
fig = go.Figure()
# ts for each store
for n_store in df['Store'].unique():
  df_store = df[df['Store'] == n_store].groupby(["Store", "Date"], as_index=False).agg({"Weekly_Sales":"mean"}).sort_values("Date")
  x_axis = df_store["Date"].unique()
  y_axis = df_store["Weekly_Sales"].tolist()
  data = go.Scatter(x=x_axis,
                    y=y_axis,
                    name=f"Store {n_store}"
                    )
  fig.add_trace(data)
  fig.update_layout(title=dict(text='weekly_sales avg for each store'))
# print
fig.show()

## dept

In [97]:
# general overview of the ts per each department
fig = go.Figure()
# ts for each store
for n_store in df['Dept'].unique():
  df_store = df[df['Dept'] == n_store].groupby(["Dept", "Date"], as_index=False).agg({"Weekly_Sales":"mean"}).sort_values("Date")
  x_axis = df_store["Date"].unique()
  y_axis = df_store["Weekly_Sales"].tolist()
  data = go.Scatter(x=x_axis,
                    y=y_axis,
                    name=f"Dept {n_store}"
                    )
  fig.add_trace(data)
  fig.update_layout(title=dict(text='weekly_sales avg for each department'))
# print
fig.show()

## holidays

In [93]:
# dataframe with holidays
df_holidays = pd.DataFrame({"Date":sorted(df.loc[df["d_is_holiday"]==1,"Date"].unique())})
df_holidays.reset_index(drop=False, inplace=True)
df_holidays["index"] = df_holidays["index"] + 1

# dataframe with days
df_dates = pd.DataFrame({"Date":sorted(df["Date"].unique())})
# merge
df_dates = df_dates.merge(df_holidays, on="Date", how="left")
df_dates["d_is_holiday"] = np.where(df_dates["Date"].isin(df_holidays["Date"]), df_dates["index"],np.nan)
df_dates["str_is_holiday"] = np.where(df_dates["d_is_holiday"].isna()==False, df_dates["Date"], "")

In [92]:
# general overview of the ts per each department
fig = go.Figure()
x_axis = df_dates["Date"].unique()
y_axis = df_dates["index"].tolist()
data = go.Scatter(x=x_axis,
                  y=y_axis,
                  name="Holidays",
                  mode="markers+text",
                  text=df_dates["str_is_holiday"],
                  textposition="bottom center"
                  )
fig.add_trace(data)
fig.update_layout(title=dict(text='Holidays'))
# print
fig.show()